# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [105]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import numpy as np


## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [106]:
# Создаем синтетический датасет
X, y = make_regression(n_samples=10000, n_features=100, noise=0.1, random_state=42)

# Разделяем датасет на тренировочную и тестовую выборки  test_size=0.2 указывает, что 20% данных будут использоваться для тестирования, а остальные 80% - для обучения модели.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [107]:
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказываем значения для тестовой выборки
y_pred = model.predict(X_test)

# Оцениваем качество модели
mse = mean_squared_error(y_test, y_pred)
print(f'среднеквадратичная ошибка {mse}')

среднеквадратичная ошибка 0.010319177035099037


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [115]:
# Создаем и обучаем модель с использованием метода градиентного спуска
model = SGDRegressor( alpha = 0.0001, random_state=42)
model.fit(X_train, y_train)

# Предсказываем значения для тестовой выборки
y_pred = model.predict(X_test)

# Оцениваем качество модели
mse = mean_squared_error(y_test, y_pred)
print(f'среднеквадратичная ошибка {mse}')

среднеквадратичная ошибка 0.010816091830096245


***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

Эта метрика измеряет, насколько средние квадратичные отклонения предсказанных значений от фактических. Для этой задачи, чем ближе значение MSE к нулю, тем лучше модель.

Различие в значениях MSE между LinearRegression и SGDRegressor может быть связано с несколькими факторами. Во-первых, гиперпараметр alpha в SGDRegressor отвечает за силу регуляризации, и его неоптимальное значение может влиять на результаты. Во-вторых, стохастический характер SGD и неявное указание количества итераций могут привести к случайным колебаниям в результатах.


***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

Для подбора гиперпараметров в методе градиентного спуска (SGDRegressor), мы можем использовать процесс подбора по сетке (Grid Search), который позволяет перебирать различные комбинации значений гиперпараметров.


In [116]:

# Определяем пространство гиперпараметров для SGDRegressor
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
}

# Создаем модель SGDRegressor
sgd_model = SGDRegressor()

# Инициируем GridSearchCV
grid_search = GridSearchCV(sgd_model, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Получаем лучшие гиперпараметры
best_params = grid_search.best_params_

# Используем лучшие гиперпараметры для обучения модели
best_sgd_model = SGDRegressor(alpha=best_params['alpha'], learning_rate=best_params['learning_rate'])
best_sgd_model.fit(X_train, y_train)
sgd_pred = best_sgd_model.predict(X_test)
sgd_mse = mean_squared_error(y_test, sgd_pred)

# Выводим результаты
print(f'Best SGDRegressor MSE: {sgd_mse} with hyperparameters: {best_params}')



Best SGDRegressor MSE: 0.01053810256554833 with hyperparameters: {'alpha': 0.0001, 'learning_rate': 'adaptive'}


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию. 

In [117]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

class MyLinearRegression:
    def __init__(self, learning_rate=0.01, max_iters=1000, tol_weight_change=1e-4, tol_mse_change=1e-4):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.tol_weight_change = tol_weight_change
        self.tol_mse_change = tol_mse_change
        self.weights = None

    def fit(self, X, y):
        if X.shape[0] != y.shape[0]:
            raise ValueError("Number of samples in X and y must be the same.")
        
        # Добавляем столбец единиц для учета свободного члена (intercept)
        X_with_intercept = np.hstack((np.ones((X.shape[0], 1)), X))
        
        # Инициализируем веса случайным образом
        self.weights = np.random.rand(X_with_intercept.shape[1])
        
        for _ in range(self.max_iters):
            # Вычисляем предсказания модели
            predictions = np.dot(X_with_intercept, self.weights)
            
            # Вычисляем ошибку предсказания
            errors = predictions - y
            
            # Вычисляем градиент функции потерь (среднеквадратичной ошибки)
            gradient = np.dot(X_with_intercept.T, errors) / len(y)
            
            # Обновляем веса с учетом градиента и скорости обучения
            new_weights = self.weights - self.learning_rate * gradient
            
            # Проверяем критерии останова
            if np.linalg.norm(new_weights - self.weights) < self.tol_weight_change:
                print("Converged: Change in weights is below tolerance.")
                break
            
            new_predictions = np.dot(X_with_intercept, new_weights)
            mse_change = np.abs(np.mean((new_predictions - y) ** 2) - np.mean((predictions - y) ** 2))
            
            if mse_change < self.tol_mse_change:
                print("Converged: Change in MSE is below tolerance.")
                break
            
            # Обновляем веса
            self.weights = new_weights
        
        else:
            print("Did not converge: Max iterations reached.")
        
    def predict(self, X):
        if X.shape[1] + 1 != len(self.weights):
            raise ValueError("Number of features in X must match the number of weights.")
        
        # Добавляем столбец единиц для учета свободного члена (intercept)
        X_with_intercept = np.hstack((np.ones((X.shape[0], 1)), X))
        return np.dot(X_with_intercept, self.weights[1:]) + self.weights[0]

# Пример использования:
# Создаем случайные данные для демонстрации
X = np.random.rand(100, 2)
y = 3 * X[:, 0] + 5 * X[:, 1] + 2 + 0.1 * np.random.randn(100)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и обучаем модель
model = MyLinearRegression(learning_rate=0.01, max_iters=1000, tol_weight_change=1e-4, tol_mse_change=1e-4)
model.fit(X_train, y_train)

# Предсказываем значения для тестовой выборки
y_pred = model.predict(X_test)

# Оцениваем качество модели
mse = mean_squared_error(y_test, y_pred)
print(f'MyLinearRegression MSE: {mse}')


Did not converge: Max iterations reached.


ValueError: shapes (20,3) and (2,) not aligned: 3 (dim 1) != 2 (dim 0)

In [118]:
my_reg = LinearRegression(regularization='l2')
my_reg.fit(X, y)
predictions = my_reg.predict(X)
print('You are amazing! Great work!')

TypeError: LinearRegression.__init__() got an unexpected keyword argument 'regularization'

***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**. 

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error

X, y = make_regression(n_samples = 10000)
# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Задаем значения параметра регуляризации
alpha_l1_1 = 0.1
alpha_l1_2 = 0.01
alpha_l2_1 = 0.1
alpha_l2_2 = 0.01

# Обучаем модели с l1-регуляризацией
lasso_model_1 = Lasso(alpha=alpha_l1_1)
lasso_model_2 = Lasso(alpha=alpha_l1_2)

lasso_model_1.fit(X_train, y_train)
lasso_model_2.fit(X_train, y_train)

# Обучаем модели с l2-регуляризацией
ridge_model_1 = Ridge(alpha=alpha_l2_1)
ridge_model_2 = Ridge(alpha=alpha_l2_2)

ridge_model_1.fit(X_train, y_train)
ridge_model_2.fit(X_train, y_train)

# Делаем предсказания
y_pred_lasso_1 = lasso_model_1.predict(X_test)
y_pred_lasso_2 = lasso_model_2.predict(X_test)

y_pred_ridge_1 = ridge_model_1.predict(X_test)
y_pred_ridge_2 = ridge_model_2.predict(X_test)

# Оцениваем качество моделей
mse_lasso_1 = mean_squared_error(y_test, y_pred_lasso_1)
mse_lasso_2 = mean_squared_error(y_test, y_pred_lasso_2)

mse_ridge_1 = mean_squared_error(y_test, y_pred_ridge_1)
mse_ridge_2 = mean_squared_error(y_test, y_pred_ridge_2)

# Выводим результаты
print("Lasso (alpha=0.1) MSE:", mse_lasso_1)
print("Lasso (alpha=0.01) MSE:", mse_lasso_2)
print("Ridge (alpha=0.1) MSE:", mse_ridge_1)
print("Ridge (alpha=0.01) MSE:", mse_ridge_2)

Lasso (alpha=0.1) MSE: 0.09844461927035866
Lasso (alpha=0.01) MSE: 0.0009842001698743407
Ridge (alpha=0.1) MSE: 5.642771301488967e-06
Ridge (alpha=0.01) MSE: 5.642904441531955e-08


***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.